# Assignment 3


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [ ]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

In [ ]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'

df = pd.read_csv(url, parse_dates = ["Date"])

In [ ]:
df.head()

In [ ]:
# examiningcolumnsand rows
df.shape

In [ ]:
# checking type for each column
df.info()

In [ ]:
# examining the names of the columns
df.columns

In [ ]:
# changing StatisticsProfileDate to datetime 
df["StatisticsProfileDate"] #= pd.to_datetime(df["StatisticsProfileDate"], errors='ignore')

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [ ]:
df['X'].unique()

In [ ]:
df['Y'].unique()

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [ ]:
df['Date'].unique()

In [ ]:
df.head()

## Melting 

In [ ]:
# Melting
id_vars = ["X","Y","Date"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="column", value_name="cases")

df.head(10)


### Dropping cases

In [ ]:
df = df[[
    "Date",
    "column",
    "cases"
    
]]

In [ ]:
df.head()

# Hospitalisations

In [ ]:
hospitalised_df = df[df['column'].str.contains("Hospitalised")]
hospitalised_df.head()

In [ ]:
hospitalised_df['column'].unique()

In [ ]:
#hospitalised_df.iloc[:,1] = hospitalised_df.iloc[:,1].str.lstrip("Hospitalised")

In [ ]:
hospitalised_df['column'].tail()

In [ ]:
hospitalised_df['column'].unique()

In [ ]:
test = "Aged5to14"

match = re.search("(\D+)(\d+)(\D{2})(\d{2})", test)
print(match[0])
print(match[1]) # Aged
print(match[2]) # 5
print(match[3]) # to
print(match[4]) # 14
print()

In [ ]:
hospitalised_df["column"]

In [ ]:
tmp_df = hospitalised_df["column"].str.extract("(\D+)(\d+)(\D{2})(\d{2})", expand=False)    


# Name columns
tmp_df.columns = ["Aged", "age_lower","to", "age_upper"]

# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

tmp_df.head(15)

In [ ]:
# Merge 
hospitalised_df = pd.concat([hospitalised_df, tmp_df], axis=1)

# Drop unnecessary columns and rows
hospitalised_df = hospitalised_df.drop(['column',"age_lower","to","age_upper" ], axis=1)
hospitalised_df = hospitalised_df.dropna()
hospitalised_df = hospitalised_df.sort_values(ascending = True,by = ["Date"])
hospitalised_df["cases"] = hospitalised_df["cases"].astype(int)



In [ ]:

hospitalised_df["Aged"] = hospitalised_df["Aged"].str.rstrip("Aged")
hospitalised_df.head(25)

# Ages

In [ ]:
aged_df = df[df["column"].str.contains("^Aged")]

In [ ]:
aged_df.head()

In [ ]:
aged_df["column"].unique()

In [ ]:
test = "Aged1to4"

match = re.search("(\D+)(\d+)(\D{2})(\d)", test)
print(match[0])
print(match[1]) # Aged
print(match[2]) # number 
print(match[3]) # to
print(match[4]) # number
print()



test = "Aged35to44"

# incorrectly drops the 
match = re.search("(\D+)(\d+)(\D{2})(\d+)", test)
print(match[0])
print(match[1]) # Aged
print(match[2]) # number 
print(match[3]) # to
print(match[4]) # number
print()



In [ ]:
tmp_df = aged_df["column"].str.extract("(\D+)(\d+)(\D{2})(\d+)", expand=False)    


# Name columns
tmp_df.columns = ["Aged", "age_lower","to", "age_upper"]

# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

tmp_df.head(15)

In [ ]:
# Merge 
aged_df = pd.concat([aged_df, tmp_df], axis=1)

# Drop unnecessary columns and rows
aged_df = aged_df.drop(['column',"age_lower","to","age_upper" ], axis=1)
aged_df = aged_df.dropna()
aged_df = aged_df.sort_values(ascending = True,by = ["Date"])
aged_df["cases"] = aged_df["cases"].astype(int)
aged_df.head(25)

### Getting the final DF

In [ ]:
# Merge 
df = pd.merge(df, aged_df, on=["Date"])
df = pd.merge(df, hospitalised_df, on=["Date"])

# Drop unnecessary columns and rows
df = df.sort_values(ascending = True,by = ["Date"])
df["cases"] = df["cases"].astype(int)
df.head(25)

In [ ]:
#df.to_csv(r"df.csv", index =  False)